## 二、填充和步幅
假设输入形状是$ n_h*n_w $，卷积核窗口形状是$ k_h*k_w $,输出形状是$ （n_h-k_h+1）*(n_w-k_w+1) $
所以卷积层的输出形状由输入形状和卷积核窗口形状决定。
卷积层的两个超参数填充和步幅,它们可以对给定形状的输入和卷积核改变输出形状。
### 2.1 填充（padding）
填充（padding）是指在输入高和宽的两侧填充元素（通常是 0 元素
如果在高的两侧一共填充 $ p_h $ 行，在宽的两侧一共填充 $ p_w $ 列，那么输出形状将会是$ (n_h−k_h+p_h+1)×(n_w−k_w+p_w+1) $ 
即输出的高和宽分别会增加 $ p_h $ 和 $ p_w $ 

在很多情况下，我们会设置 $ p_h=k_h−1 $ 和 $ p_w=k_w−1 $ 使得输入和输出具有相同的高和宽。这样会方便在构造网络时推测每个层的输出形状。 

假设这里 $ k_h $ 是奇数，我们会在高的两侧分别填充 $ p_h/2 $ 行。如果 $ k_h $ 是偶数，一种可能是在输入的顶端一侧填充 $ ⌈p_h/2⌉ $ 行，而在底端一侧填充 $ ⌊p_h/2⌋ $ 行。在宽的两侧填充同理。


In [4]:
from mxnet import nd
from mxnet.gluon import nn

# 定义一个便利函数来计算卷积层。它初始化卷积层权重，并对输入核输出做相应的v升维和降维
def comp_conv2d(conv2d,X):
    conv2d.initialize()
    # （1，1）代表批量大小和通道数，均为1
    print('X.shape:',X.shape)
    X=X.reshape((1,1)+X.shape)
    print('X.shape:',X.shape)
    Y=conv2d(X)
    return Y.reshape(Y.shape[2:]) # 排除不关心的前两维：批量和通道

# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
conv2d=nn.Conv2D(1,kernel_size=3,padding=1)
X=nd.random.uniform(shape=(8,8))
comp_conv2d(conv2d,X).shape

X.shape: (8, 8)
X.shape: (1, 1, 8, 8)


(8, 8)

In [6]:
# 使用高为 5、宽为 3 的卷积核。在高和宽两侧的填充数分别为 2 和 1。
conv2d=nn.Conv2D(1,kernel_size=(5,3),padding=(2,1))
comp_conv2d(conv2d,X).shape

X.shape: (8, 8)
X.shape: (1, 1, 8, 8)


(8, 8)

### 2.2 步幅（stride）
二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。
我们将每次滑动的行数和列数称为步幅。

一般来说，当高上步幅为 $ s_h $，宽上步幅为 $ s_w $ 时，输出形状为
$ ⌊(n_h−k_h+p_h+s_h)/s_h⌋×⌊(n_w−k_w+p_w+s_w)/s_w⌋ $.

如果我们设置 $ p_h=k_h−1 $ 和 $ p_w=k_w−1 $，那么输出形状将简化为 $ ⌊(n_h+s_h−1)/s_h⌋×⌊(n_w+s_w−1)/s_w⌋ $
更进一步，如果输入的高和宽能分别被高和宽上的步幅整除，那么输出形状将是 $ (n_h/s_h)×(n_w/s_w) $



In [8]:
conv2d=nn.Conv2D(1,kernel_size=3,padding=1,strides=2)
comp_conv2d(conv2d,X).shape

X.shape: (8, 8)
X.shape: (1, 1, 8, 8)


(4, 4)

In [9]:
conv2d=nn.Conv2D(1,kernel_size=(3,5),padding=(0,1),strides=(3,4))
comp_conv2d(conv2d,X).shape

X.shape: (8, 8)
X.shape: (1, 1, 8, 8)


(2, 2)